In [1]:
import json
from collections import Counter
import re
from text_utils import remove_reg
from tqdm.notebook import tqdm
import pickle

from wiki_labels import qid_lab_get
qid_lab_get(42, 'he')

{'דאגלס אדמס': {'he'}}

In [ ]:
# PAC assess fn ln order
fnln_order = Counter()
fo =  open('/backup/wikidata/fnln_order.txt', 'w')
for l in tqdm(open('/backup/wikidata/wiki_person.jsonl'), total=10_044_572):
    j = json.loads(l)
    qid = int(j['id'][1:])
    fnid = int(j['fn'][0][6:]) if len(j['fn']) > 0 else None
    lnid = int(j['ln'][0][6:]) if len(j['ln']) > 0 else None
    labs = qid_lab_get(qid, include_alt=False)
    if fnid:
        fns = qid_lab_get(fnid, include_alt=True)
        fns = [remove_reg.sub(' ', fn) for fn in fns.keys()]
        fn_re = re.compile('^'+'|'.join(fns)+'.+')
        for n, langs in labs.items():
            if len(langs) == 1 and fn_re.match(n) and ',' not in n:
                for lang in langs:
                    fnln_order[(lang, 'f')] += 1
                    fo.write(f'fl\t{n}\t{lang}\n')
    if lnid:
        lns = qid_lab_get(lnid, include_alt=True)
        lns = [remove_reg.sub(' ', ln) for ln in lns.keys()]
        ln_re = re.compile('^'+'|'.join(lns)+'.+')
        for n, langs in labs.items():
            if len(langs) == 1 and ln_re.match(n) and ',' not in n:
                for lang in langs:
                    fnln_order[(lang, 'l')] += 1
                    fo.write(f'lf\t{n}\t{lang}\n')

pickle.dump(fnln_order, open('/backup/wikidata/fnln_order.pickle', 'wb'))


  0%|          | 0/10044572 [00:00<?, ?it/s]

In [4]:
#l = 'th'
#fnln_order[(l, 'l')] / (fnln_order[(l, 'l')]+fnln_order[(l, 'f')])
qid, fns

(43661,
 ['==)à',
  'dimtri',
  'dmitr',
  'dmitri',
  'dmitry',
  "d'mitr",
  'demitri',
  'dimitri',
  'dimitry',
  'dmitrij',
  'dmitriy',
  'dmitrii',
  'митя',
  'дима',
  'dmitrijs',
  'дми́трий',
  'demetrius',
  'димочка',
  'димка',
  'дмітрій',
  'зьміцер',
  '德米特里',
  'ドミトリ',
  'ντμίτρι',
  'дмитро',
  'ντιμίτρι',
  'дзьмітры',
  'дзмітрый',
  'дмитрий',
  'დიმიტრი',
  'ديميتري',
  'ディミトリー'])